# Metadata for the National Flood Insurance Program (NFIP) Claims and Policies Datasets
Author: Mark Bauer

In [1]:
# import libraries
from datetime import datetime
import json
import requests
import pandas as pd
import duckdb

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p json,requests,pandas,duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

json    : 2.0.9
requests: 2.28.1
pandas  : 1.5.1
duckdb  : 1.0.0



In [3]:
# data retrieved, not to be confused with data of data
current_date = datetime.now()
print(f"Metadata was retrieved on {current_date.strftime('%Y-%m-%d')}.")

Metadata was retrieved on 2025-05-17.


# OpenFEMA Metadata Endpoints
This notebook focuses on using two key OpenFEMA API endpoints:

- OpenFemaDataSets – a list of available FEMA datasets

- OpenFemaDataSetFields – details about the fields in each dataset

Finally, we save the data dictionaries as CSV files.

More information about the APIs can be found in the [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api).

# OpenFemaDataSets

In [4]:
# API without dataset filter, all datasts
url = "https://www.fema.gov/api/open/v1/OpenFemaDataSets"
response = requests.get(url)

if response.ok:
    data = response.json()
    print("Top-level keys:", list(data.keys()))
else:
    print(f"Failed to download data. Status code: {response.status_code}")

Top-level keys: ['metadata', 'OpenFemaDataSets']


In [5]:
# preview value type
type(data.get('OpenFemaDataSets', {}))

list

In [6]:
# preview first item in list
data.get('OpenFemaDataSets', {})[0]

{'identifier': 'openfema-26',
 'name': 'FemaWebDeclarationAreas',
 'title': 'FEMA Web Declaration Areas',
 'description': "This data set contains general information on declared disasters, including the disaster number, declaration type, state, description, incident start and end dates, and incident type.\n\nIf you have media inquiries about this dataset, please email the FEMA Press Office at FEMA-Press-Office@fema.dhs.gov or call (202) 646-3272. For inquiries about FEMA's data and Open Government program, please email the OpenFEMA team at OpenFEMA@fema.dhs.gov.",
 'webService': 'https://www.fema.gov/api/open/v1/FemaWebDeclarationAreas',
 'dataDictionary': 'https://www.fema.gov/openfema-data-page/fema-web-declaration-areas-v1',
 'keyword': ['disaster', 'declaration', 'fema website'],
 'modified': '2023-01-10T00:00:00.000Z',
 'publisher': 'Federal Emergency Management Agency',
 'contactPoint': 'OpenFEMA',
 'mbox': 'openfema@fema.gov',
 'accessLevel': 'public',
 'landingPage': 'https://w

In [7]:
# number of datasets
len(data.get('OpenFemaDataSets', {}))

43

In [8]:
# create dataframe
datasets_df = pd.DataFrame(data.get('OpenFemaDataSets', {}))

datasets_df.head()

,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,...,recordCount,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
0,openfema-26,FemaWebDeclarationAreas,FEMA Web Declaration Areas,This data set contains general information on ...,https://www.fema.gov/api/open/v1/FemaWebDeclar...,https://www.fema.gov/openfema-data-page/fema-w...,"[disaster, declaration, fema website]",2023-01-10T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,473983.0,None,None,None,None,dc72e3569e7cbb8fdbeaaf404672cab4e322328d,2024-11-21T14:40:15.168Z,3f2975a6-3c3b-4c29-9ad0-3954efab268d,2025-05-17T14:21:44.063Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
1,openfema-24,FemaWebDisasterSummaries,FEMA Web Disaster Summaries,This data set contains financial assistance va...,https://www.fema.gov/api/open/v1/FemaWebDisast...,https://www.fema.gov/openfema-data-page/fema-w...,"[disaster, declarations, fema website, disaste...",2023-01-10T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,3827.0,None,None,None,None,58850b3b357cc8d55796a420ce7fbf570a769872,2024-11-21T14:40:15.168Z,2312fe52-46fc-4930-aa57-9a52cb25813b,2025-05-17T14:21:55.022Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
2,openfema-44,FemaRegions,FEMA Regions,Provides a list of FEMA Regions including the ...,https://www.fema.gov/api/open/v2/FemaRegions,https://www.fema.gov/openfema-data-page/fema-r...,[regions],2023-09-26T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,NaN,None,None,None,None,7329337cb152bbcbfc884bf7dad14f3d59cddba0,2024-11-21T14:40:15.168Z,912bf1d5-be86-4536-98d1-5a093e7bab42,None,"[{'format': 'csv', 'accessURL': 'https://www.f..."
3,openfema-2,PublicAssistanceFundedProjectsDetails,Public Assistance Funded Projects Details,The Public Assistance Funded Projects Details ...,https://www.fema.gov/api/open/v1/PublicAssista...,https://www.fema.gov/openfema-data-page/public...,"[public, assistance, mitigation, grants, sub-g...",2025-03-05T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,808460.0,None,None,None,None,9dd2ed7f062a46fca906e5b83556688b735c5f1e,2025-03-05T19:40:34.218Z,a7690da5-2f19-4ca5-87fc-3257cc042e17,2025-05-16T15:03:49.891Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
4,openfema-35,IpawsArchivedAlerts,IPAWS Archived Alerts,The Integrated Public Alert and Warning System...,https://www.fema.gov/api/open/v1/IpawsArchived...,https://www.fema.gov/openfema-data-page/ipaws-...,"[IPAWS, alerts, cap, alert, warning, public al...",2024-05-30T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,4861648.0,None,None,None,None,d06e1de57e5dee74cddb4de598a513195fcb94e4,2024-11-21T14:40:15.168Z,e942cf14-96dd-4925-992a-35ceac16990e,2025-05-17T14:34:25.643Z,"[{'format': 'json', 'accessURL': 'https://www...."


In [9]:
# column info
datasets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   identifier              43 non-null     object 
 1   name                    43 non-null     object 
 2   title                   43 non-null     object 
 3   description             43 non-null     object 
 4   webService              43 non-null     object 
 5   dataDictionary          43 non-null     object 
 6   keyword                 43 non-null     object 
 7   modified                43 non-null     object 
 8   publisher               43 non-null     object 
 9   contactPoint            43 non-null     object 
 10  mbox                    43 non-null     object 
 11  accessLevel             43 non-null     object 
 12  landingPage             42 non-null     object 
 13  temporal                34 non-null     object 
 14  api                     43 non-null     bool

In [10]:
# slice through columns to preview all of them
datasets_df.iloc[:5, :20]

,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,mbox,accessLevel,landingPage,temporal,api,version,bureauCode,programCode,accessLevelComment,license
0,openfema-26,FemaWebDeclarationAreas,FEMA Web Declaration Areas,This data set contains general information on ...,https://www.fema.gov/api/open/v1/FemaWebDeclar...,https://www.fema.gov/openfema-data-page/fema-w...,"[disaster, declaration, fema website]",2023-01-10T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/disasters,1960-11-01/,True,1,[024:70],[024:039],None,None
1,openfema-24,FemaWebDisasterSummaries,FEMA Web Disaster Summaries,This data set contains financial assistance va...,https://www.fema.gov/api/open/v1/FemaWebDisast...,https://www.fema.gov/openfema-data-page/fema-w...,"[disaster, declarations, fema website, disaste...",2023-01-10T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/disasters,1998-08-26/,True,1,[024:70],[024:039],None,None
2,openfema-44,FemaRegions,FEMA Regions,Provides a list of FEMA Regions including the ...,https://www.fema.gov/api/open/v2/FemaRegions,https://www.fema.gov/openfema-data-page/fema-r...,[regions],2023-09-26T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,None,None,True,2,[024:70],[024:039],None,None
3,openfema-2,PublicAssistanceFundedProjectsDetails,Public Assistance Funded Projects Details,The Public Assistance Funded Projects Details ...,https://www.fema.gov/api/open/v1/PublicAssista...,https://www.fema.gov/openfema-data-page/public...,"[public, assistance, mitigation, grants, sub-g...",2025-03-05T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/assistance/public,1980-02-01/,True,1,[024:70],[024:039],None,None
4,openfema-35,IpawsArchivedAlerts,IPAWS Archived Alerts,The Integrated Public Alert and Warning System...,https://www.fema.gov/api/open/v1/IpawsArchived...,https://www.fema.gov/openfema-data-page/ipaws-...,"[IPAWS, alerts, cap, alert, warning, public al...",2024-05-30T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/emergency-managers/practi...,2012-06-01/,True,1,[024:70],[024:039],None,None


In [11]:
datasets_df.iloc[:5, 20:]

,spatial,theme,dataQuality,accrualPeriodicity,language,primaryITInvestmentUII,references,issued,systemOfRecords,recordCount,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
0,None,Disaster Information,true,R/PT20M,en-US,None,[],2013-05-01T00:00:00.000Z,None,473983.0,None,None,None,None,dc72e3569e7cbb8fdbeaaf404672cab4e322328d,2024-11-21T14:40:15.168Z,3f2975a6-3c3b-4c29-9ad0-3954efab268d,2025-05-17T14:21:44.063Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
1,None,Disaster Information,true,R/PT20M,en-US,None,[],2012-01-01T00:00:00.000Z,None,3827.0,None,None,None,None,58850b3b357cc8d55796a420ce7fbf570a769872,2024-11-21T14:40:15.168Z,2312fe52-46fc-4930-aa57-9a52cb25813b,2025-05-17T14:21:55.022Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
2,None,Miscellaneous,true,irregular,en-US,None,[],2017-01-01T00:00:00.000Z,None,NaN,None,None,None,None,7329337cb152bbcbfc884bf7dad14f3d59cddba0,2024-11-21T14:40:15.168Z,912bf1d5-be86-4536-98d1-5a093e7bab42,None,"[{'format': 'csv', 'accessURL': 'https://www.f..."
3,None,Public Assistance,true,R/P1D,en-US,None,[],2010-07-15T00:00:00.000Z,None,808460.0,None,None,None,None,9dd2ed7f062a46fca906e5b83556688b735c5f1e,2025-03-05T19:40:34.218Z,a7690da5-2f19-4ca5-87fc-3257cc042e17,2025-05-16T15:03:49.891Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
4,None,Emergency Management/Preparedness/Alerts,true,R/PT2M,en-US,None,[https://docs.oasis-open.org/emergency/cap/v1....,2020-03-27T00:00:00.000Z,None,4861648.0,None,None,None,None,d06e1de57e5dee74cddb4de598a513195fcb94e4,2024-11-21T14:40:15.168Z,e942cf14-96dd-4925-992a-35ceac16990e,2025-05-17T14:34:25.643Z,"[{'format': 'json', 'accessURL': 'https://www...."


In [12]:
# number of datasets per theme
datasets_df['theme'].value_counts()

Hazard Mitigation                           10
National Flood Insurance Program             8
Individual Assistance                        7
Disaster Information                         6
Public Assistance                            5
Miscellaneous                                4
Emergency Management/Preparedness/Alerts     3
Name: theme, dtype: int64

In [13]:
# preview NFIP datasets
datasets_df.loc[datasets_df['theme'] == 'National Flood Insurance Program']

,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,...,recordCount,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
13,openfema-74,FimaNfipPolicies,FIMA NFIP Redacted Policies,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipPolicies,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, Flood, Insurance, Policy, Water]",2025-04-01T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,69829956.0,None,None,None,None,85c6b91e237853b3333b06b4e3ebf5cdef930060,2025-04-01T20:00:37.246Z,3776bead-efe2-4590-84eb-3e000f869701,2025-05-14T13:43:49.278Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
14,openfema-77,NfipCommunityLayerComprehensive,NFIP Community Layer Comprehensive,This dataset is also known as the 3d layer and...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,51943.0,None,None,None,None,51551de69709270cbb3dc05a9da88e92be099c2d,2024-12-05T22:20:23.322Z,0131d365-5a18-47a0-98cb-558149c165ee,2024-12-05T18:09:51.898Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
15,openfema-73,FimaNfipClaims,FIMA NFIP Redacted Claims,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipClaims,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, FIMA, claims, flood insurance]",2023-03-24T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,2712269.0,None,None,None,None,843979427d2c4cc20e139ce661e6b0476975431f,2024-11-21T14:40:15.168Z,e9343760-d76e-44db-b947-822642353e7f,2025-05-14T13:01:34.036Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
16,openfema-86,NfipResidentialPenetrationRates,NFIP Residential Penetration Rates,"NFIP ""take-up rates"" or residential penetratio...",https://www.fema.gov/api/open/v1/NfipResidenti...,https://www.fema.gov/openfema-data-page/nfip-r...,"[nfip, rates, nsi, residential structures, pol...",2024-11-07T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,3145.0,None,None,None,None,c84151b7bbe46b75c5143a0c9d491a9986247f06,2024-11-21T14:40:15.168Z,18f6002e-40a5-4601-9504-279b8e6bf985,2025-01-02T23:01:15.547Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
17,openfema-76,NfipCommunityLayerNoOverlapsSplit,NFIP Community Layer No Overlaps Split,This dataset is flattened and multicounty comm...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,37918.0,None,None,None,None,8db6fcd098f2b2bf16a3dc6b10ccb93543d32010,2024-12-05T22:20:23.322Z,57d33198-ea17-4255-83fe-35742b8f601b,2024-12-05T18:09:57.312Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
18,openfema-75,NfipCommunityLayerNoOverlapsWhole,NFIP Community Layer No Overlaps Whole,This dataset is flattened and multicounty comm...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,29898.0,None,None,None,None,7bdabebd0a0805adbb76fee3ca61bbd4a5f5c0b7,2024-12-05T22:20:23.322Z,e2c2268c-7a3c-4ce1-830e-ac5f735ee44c,2024-12-05T18:10:50.474Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
19,openfema-39,NfipCommunityStatusBook,NFIP Community Status Book,The National Flood Insurance Program (NFIP) en...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[NFIP, Status Book, Community Status Book, Flo...",2024-11-07T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,25053.0,None,None,None,None,7b8f6d05763d0c656c524c44438e1d8e8552abe8,2024-11-21T14:40:15.168Z,edccafe8

We use a filter option to only return metadata about the NFIP Claims and Policies datasets.

In [14]:
url = (
    "https://www.fema.gov/api/open/v1/OpenFemaDataSets"
    "?$filter=name eq 'FimaNfipClaims' or name eq 'FimaNfipPolicies'"
)

response = requests.get(url)

if response.ok:
    data = response.json()
    print("Top-level keys:", list(data.keys()))
    print("Dataset names:", [d["name"] for d in data.get("OpenFemaDataSets", [])])
else:
    print(f"Failed to download data. Status code: {response.status_code}")

Top-level keys: ['metadata', 'OpenFemaDataSets']
Dataset names: ['FimaNfipPolicies', 'FimaNfipClaims']


In [15]:
print(json.dumps(data.get('metadata', {}), indent=2))

{
  "skip": 0,
  "filter": "name eq 'FimaNfipClaims' or name eq 'FimaNfipPolicies'",
  "orderby": "",
  "select": null,
  "rundate": "2025-05-17T14:41:37.535Z",
  "top": 1000,
  "format": "json",
  "metadata": true,
  "entityname": "OpenFemaDataSets",
  "version": "v1",
  "url": "/api/open/v1/OpenFemaDataSets?$filter=name%20eq%20'FimaNfipClaims'%20or%20name%20eq%20'FimaNfipPolicies'",
  "count": 0
}


In [16]:
# preview to check what's returned, get a feel for the datasets
print(json.dumps(data.get('OpenFemaDataSets', {}), indent=2))

[
  {
    "identifier": "openfema-74",
    "name": "FimaNfipPolicies",
    "title": "FIMA NFIP Redacted Policies",
    "description": "Congress passed the National Flood Insurance Act (NFIA), 42 U.S.C. 4001 in 1968, creating the National Flood Insurance Program (NFIP) in order to reduce future flood losses through flood hazard identification, floodplain management, and providing insurance protection. The Department of Housing and Urban Development (HUD) originally administered the NFIP, and Congress subsequently transferred the NFIP to FEMA upon its creation in 1979. FEMA and insurance companies participating in FEMA's Write Your Own (WYO) program offer NFIP insurance coverage for building structures as well as for contents and personal property within the building structures to eligible and insurable properties. The WYO program began in 1983 with NFIP operating under Part B of the NFIA and allows FEMA to authorize private insurance companies to issue the Standard Flood Insurance Polic

# OpenFemaDataSetFields
We use the filter option to directly return the NFIP Claims dataset information.

In [17]:
url = "https://www.fema.gov/api/open/v1/OpenFemaDataSetFields?$filter=openFemaDataSet%20eq%20%27FimaNfipClaims%27%20and%20datasetVersion%20eq%202"
response = requests.get(url)

if response.ok:
    data = response.json()
    print("Top-level keys:", list(data.keys()))
else:
    print(f"Failed to download data. Status code: {response.status_code}")

Top-level keys: ['metadata', 'OpenFemaDataSetFields']


In [18]:
# create a dataframe
claims_df = pd.DataFrame(data.get('OpenFemaDataSetFields', {}))

# preview number columns, rows
print(claims_df.shape)
claims_df

(73, 16)


,datasetId,openFemaDataSet,datasetVersion,name,title,description,type,sortOrder,isSearchable,isNestedObject,isNullable,primaryKey,id,lastRefresh,hash,srid
0,openfema-73,FimaNfipClaims,2,asOfDate,As of Date,The effective date of the data in the file. In...,datetime,2,True,False,True,False,97719ad8-6820-46ef-baca-5ac81be4ce30,2024-11-07T21:42:54.493Z,5ebd8717745887e4f21f71f3107339b807b7a75b,None
1,openfema-73,FimaNfipClaims,2,amountPaidOnBuildingClaim,Amount Paid on Building Claim,Dollar amount paid on the building claim. In s...,"decimal(12,2)",22,True,False,True,False,e2200fb7-0b8b-439f-b8f1-020c3348c1f5,2025-05-14T13:30:03.484Z,d5c21037158c9371f42685900309d4f66e6d2517,None
2,openfema-73,FimaNfipClaims,2,amountPaidOnContentsClaim,Amount Paid on Contents Claim,Dollar amount paid on the contents claim. In s...,"decimal(12,2)",23,True,False,True,False,da12f176-de08-4bbd-bb6d-61b89ea6eebd,2025-05-14T13:30:03.484Z,8966a6fbcea3ae1028fae6a3740178a6e54731ad,None
3,openfema-73,FimaNfipClaims,2,amountPaidOnIncreasedCostOfComplianceClaim,Amount Paid on Increased Cost of Compliance (I...,ICC coverage is one of several flood insurance...,"decimal(12,2)",24,True,False,True,False,9d2f03fe-fb09-48f2-ac16-136d8ce1e73f,2025-05-14T13:30:03.484Z,6081a92a1c9febd5b50cfa07f885b8cb692ce29d,None
4,openfema-73,FimaNfipClaims,2,netBuildingPaymentAmount,Net Building Payment Amount,Net building payment amount made to insured in...,"decimal(12,2)",34,True,False,True,False,52278c05-990b-41e2-a779-b26915a2852d,2025-05-14T13:30:03.484Z,30d68fd2223ef8b8a0fbcec3698d769d96c40be8,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,openfema-73,FimaNfipClaims,2,censusTract,Census Tract,U.S. Census Bureau defined census tracts; stat...,text,69,True,False,True,False,b7948f15-f64e-406e-8028-4f6da8c8f48f,2024-08-27T19:21:04.454Z,0f5a536267c5f8a00dacd63ab1d8c71731728d64,None
69,openfema-73,FimaNfipClaims,2,censusBlockGroupFips,Census Block Group FIPS,The first digit of the census block number ide...,text,70,True,False,True,False,b5b80826-065e-44aa-98ee-65a8a3aa3c99,2024-08-27T19:21:04.454Z,79c1dde3b0b137dfa8eebed442b97951e440bbb2,None
70,openfema-73,FimaNfipClaims,2,latitude,Latitude,Approximate latitude of the insured building (...,"decimal(9,1)",71,True,False,True,False,a3ffbbc7-2af4-47e8-8406-19aa0038fefa,2024-08-27T19:21:04.454Z,0a546b9587f2617a3bdee20255bfa3ec20bcbb1c,None
71,openfema-73,FimaNfipClaims,2,longitude,Longitude,Approximate longitude of the insured building ...,"decimal(9,1)",72,True,False,True,False,5f36c9c0-7ca2-44ab-843a-c251f872bdb2,2024-08-27T19:21:04.454Z,245fe176844bc4300e45ca404a1ec9909f694eb9,None


Now we use the filter option to directly return the NFIP Policies dataset information.

In [19]:
url = "https://www.fema.gov/api/open/v1/OpenFemaDataSetFields?$filter=openFemaDataSet%20eq%20%27FimaNfipPolicies%27%20and%20datasetVersion%20eq%202"
response = requests.get(url)

if response.ok:
    data = response.json()
    print("Top-level keys:", list(data.keys()))
else:
    print(f"Failed to download data. Status code: {response.status_code}")

Top-level keys: ['metadata', 'OpenFemaDataSetFields']


In [20]:
# create a dataframe
policies_df = pd.DataFrame(data.get('OpenFemaDataSetFields', {}))

# preview number columns, rows
print(policies_df.shape)
policies_df

(81, 16)


,datasetId,openFemaDataSet,datasetVersion,name,title,description,type,sortOrder,isSearchable,isNestedObject,isNullable,primaryKey,id,lastRefresh,hash,srid
0,openfema-74,FimaNfipPolicies,2,buildingReplacementCost,Building Replacement Cost,Estimated cost in whole dollars to replace the...,bigint,45,True,False,True,False,3655e95f-3b94-4dcc-b99a-009305528b7f,2024-11-06T18:42:40.839Z,0975fb5c25f813d9c0d5167091f6fe989f445fa6,None
1,openfema-74,FimaNfipPolicies,2,baseFloodElevation,Base Flood Elevation,Base Flood Elevation (BFE) is the elevation in...,"decimal(8,2)",2,True,False,True,False,23999c6f-3bb3-43bc-bb7d-60210369d501,2025-05-14T13:30:03.484Z,ab87f22d05b87fa010ce74d4d66f09c67cf8cef2,None
2,openfema-74,FimaNfipPolicies,2,lowestAdjacentGrade,Lowest Adjacent Grade,Lowest natural grade adjacent to the insured s...,"decimal(8,2)",18,True,False,True,False,f5420d25-12de-437a-8135-c9a53e72a2d7,2025-05-14T13:30:03.484Z,9eb780d9c0521aa253585a754603d1a11b7125b0,None
3,openfema-74,FimaNfipPolicies,2,lowestFloorElevation,Lowest Floor Elevation,A building's lowest floor is the floor or leve...,"decimal(8,2)",19,True,False,True,False,11e2e535-40fa-4b2a-a9ac-28bfcc434568,2025-05-14T13:30:03.484Z,83d60bc32dd0d309a4fb48bfe7ea1894daddeb6e,None
4,openfema-74,FimaNfipPolicies,2,cancellationVoidanceReasonCode,Cancellation/Voidance Reason Code,Reason code indicating why a policy was cancel...,text,39,True,False,True,False,72b03d9e-5eb4-4f3d-9b9b-f6f1341206d4,2025-05-14T13:30:03.484Z,324830cc164115e8ca2c60b3269deb347a54ab6a,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,openfema-74,FimaNfipPolicies,2,censusTract,Census Tract,US Census Bureau defined census Tracts; statis...,text,77,True,False,True,False,551a8801-003e-46c2-964f-141235880be7,2024-08-27T19:21:04.454Z,07725faae0683845ae63614c68bb8dec01aff5d0,None
77,openfema-74,FimaNfipPolicies,2,censusBlockGroupFips,Census Block Group FIPS,The first digit of the census block number ide...,text,78,True,False,True,False,d832e2d2-158a-4e15-a7b6-cdbfcca278ca,2024-08-27T19:21:04.454Z,24802d5d5dd0e512bf0b730d4784d0ab3aed738b,None
78,openfema-74,FimaNfipPolicies,2,latitude,Latitude,Approximate latitude of the insured building (...,"decimal(9,1)",79,True,False,True,False,5bd3aa38-e6dc-45ce-ae00-d57138649f10,2024-08-27T19:21:04.454Z,216c17b78be2010f499f1f2401b4c74afbde3173,None
79,openfema-74,FimaNfipPolicies,2,longitude,Longitude,Approximate longitude of the insured building ...,"decimal(9,1)",80,True,False,True,False,9332e621-d564-4d0d-afd7-c41008e9ba71,2024-08-27T19:21:04.454Z,f9591782c9dc1526753f42f33baae1c4899c652b,None


# Write Out Data Dictionaries as CSVs

In [21]:
claims_df.to_csv("data-dictionary-claims.csv", index=False)
policies_df.to_csv("data-dictionary-policies.csv", index=False)

# sanity check
%ls

LICENSE                       data-dictionary-policies.csv
README.md                     download-data.ipynb
analysis-claims.ipynb         figures/
analysis-policies.ipynb       figures.ipynb
data/                         metadata.ipynb
data-dictionary-claims.csv
